# Deep Learning Regression Model for Predicting chances of applicant being admitted
---

- Author: [Stefanus Bernard Melkisedek](https://www.github.com/stefansphtr)
- Email: [stefanussipahutar@gmail.com](stefanussipahutar@gmail.com)
- Date: 2024-02-04

## Project Description

This project is a regression model that predicts the chances of an applicant being admitted to a university based on their scores in two exams. The dataset contains nine columns: 

1. Serial No. (index)
2. GRE Score (int)
3. TOEFL Score (int)
4. University Rating (int) 
5. SOP (Statement of Purpose) (float)
6. LOR (Letter of Recommendation) (float)
7. CGPA (Cumulative Grade Point Average) (float)
8. Research (int)
9. Chance of Admit (float)

The model is implemented using a neural network with two hidden layers. The model is trained on the training set and evaluated on the test set. The performance of the model is evaluated using the mean squared error (MSE) and the R-squared score.

## Prepare the libraries

In [ ]:
# Importing libraries for data manipulation and analysis
import numpy as np  # For numerical operations
import pandas as pd  # For data manipulation and analysis
import matplotlib.pyplot as plt  # For data visualization

# Importing TensorFlow and Keras for creating and training the neural network model
import tensorflow as tf  # For machine learning and numerical computation
from tensorflow import keras  # High-level API to build and train models in TensorFlow
from keras.models import Sequential  # For linear stacking of layers
from keras.callbacks import EarlyStopping  # To stop training when a monitored metric has stopped improving
from keras.layers import Dense  # For fully connected layers

# Importing Scikit-learn libraries for data preprocessing and performance metrics
from sklearn.model_selection import train_test_split  # For splitting the data into train and test sets
from sklearn.preprocessing import StandardScaler  # For standardization of features
from sklearn.preprocessing import Normalizer  # For normalization of features
from sklearn.metrics import r2_score  # For regression performance metrics